In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Data Preprocessing

In [2]:
data = pd.read_csv('data.csv')


In [3]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

## Data Encoding

In [4]:
X_encoded = pd.get_dummies(X, columns=['Gender'])

In [5]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

## Standard Feature Scaling

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

## ANOVA F-Value Feature Selection

In [7]:
k = 5
selector = SelectKBest(score_func=f_classif, k=k)
X_selected = selector.fit_transform(X_scaled, y_encoded)

## Training and Test Data Allocation

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_encoded, test_size=0.2, random_state=42)


# Model Training and Prediction

## Hyperparameter Optimization

In [9]:
# param_grid = [
#     {   'C': [0.001, 0.01, 0.1, 1, 10, 100],
#         'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
# ]

# optimal_params = GridSearchCV(
#     MLP(),
#     param_grid,
#     cv = 5,
#     scoring = 'accuracy',
#     verbose = 0
# )

In [10]:
# optimal_params.fit(X_train, y_train)
# print(optimal_params.best_params_)

## Training

In [11]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 50, 25), activation='tanh', alpha=0.01)
mlp.fit(X_train, y_train)

c:\Users\mattm\anaconda3\envs\compbio\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='tanh', alpha=0.01, hidden_layer_sizes=(100, 50, 25))

## Prediction

In [12]:
y_pred = mlp.predict(X_test)

# Results

## Decoding

In [13]:
y_pred_decoded = label_encoder.inverse_transform(y_pred)
y_test_decoded = label_encoder.inverse_transform(y_test)

## Confusion Matrix

In [14]:
# cm = confusion_matrix(y_test, prediction)
# cm_fig = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ['ND', 'PD', 'D'])
# cm_fig.plot()
# plt.show()

In [15]:
print("Confusion Matrix:")
print(confusion_matrix(y_test_decoded, y_pred_decoded))


Confusion Matrix:
[[ 21   0   0]
 [  0   6   0]
 [  0   1 172]]


## Classification Report

In [16]:
print(classification_report(y_test_decoded, y_pred_decoded))

              precision    recall  f1-score   support

           N       1.00      1.00      1.00        21
           P       0.86      1.00      0.92         6
           Y       1.00      0.99      1.00       173

    accuracy                           0.99       200
   macro avg       0.95      1.00      0.97       200
weighted avg       1.00      0.99      1.00       200

